In [24]:
import pandas as pd
import numpy as np


In [25]:
# Load data
projections = pd.read_csv('your_file_cleaned.csv').dropna(subset=['Player'])
adp = pd.read_csv('fantasy_players_from_paste.csv').dropna(subset=['Player'])
s4_points = pd.read_csv('2024_s.csv').dropna(subset=['Player'])


In [26]:
# Clean player names
for df in [projections, adp, s4_points]:
    df['Player'] = df['Player'].str.strip()


In [27]:
# Get user input
num_players = int(input('How many players do you wish to analyze?: '))
player_analyzed = []
inherited_pick = []


How many players do you wish to analyze?:  1


In [28]:

for i in range(num_players):
    player_analyzed.append(input(f'Player {i + 1}: ').strip())
    inherited_pick.append(float(input(f'Inherited Pick {i + 1}: ')))


Player 1:  George Kittle
Inherited Pick 1:  5.4


In [29]:
# Create input DataFrame
input_data = pd.DataFrame({
    'Player': player_analyzed,
    'Inherited Pick': inherited_pick
})


In [30]:

# Safe merge function
def safe_merge(left, right, on_col='Player'):
    right = right.drop_duplicates(subset=[on_col])
    return pd.merge(left, right, on=on_col, how='left')


In [31]:
# Perform all merges
merged = safe_merge(input_data, projections)
merged = safe_merge(merged, adp)
merged = safe_merge(merged, s4_points)


In [32]:
# Fill missing values
merged['ADP'] = merged['ADP'].fillna(999)
merged['Projected_Points'] = merged['Projected_Points'].fillna(0)


In [33]:
# Calculate position ranks, I need to calculate posiiton scarcity, how doe the top 3 of the position
#compare with the rest, if there is notable difference, reward, if not, keep same
position_projected = safe_merge(adp, projections)
position_projected['PosRank'] = position_projected.groupby('Position')['Projected_Points'].rank(ascending=False)
position_projected['PosCount'] = position_projected.groupby('Position')['Projected_Points'].transform('count')

print(position_projected)



                       Player Position Team   ADP  MarketIndex   Opp    GP  \
0               Ja'Marr Chase       WR  CIN   1.1         -3.0  @CLE  16.0   
1              Saquon Barkley       RB  PHI   1.2          1.0   DAL  15.0   
2              Bijan Robinson       RB  ATL   1.3          1.0    TB  15.0   
3            Justin Jefferson       WR  MIN   1.4         -4.0  @CHI  16.0   
4                Jahmyr Gibbs       RB  DET   1.5         -1.0   @GB  15.0   
..                        ...      ...  ...   ...          ...   ...   ...   
295      Jacksonville Jaguars      DEF  JAC  30.6        -97.0   NaN   NaN   
296            Khalil Herbert       RB  IND  30.7          NaN   MIA  16.0   
297        New Orleans Saints      DEF   NO  30.8          NaN   NaN   NaN   
298  Marquez Valdes-Scantling       WR  SEA  30.9          NaN    SF  16.0   
299              Kirk Cousins       QB  ATL  30.1          NaN    TB   1.0   

    Passing_Yards Passing_TD Passing_Int Rushing_Yards  Project

In [34]:

# Merge ranks back
merged = safe_merge(merged, position_projected[['Player', 'PosRank', 'PosCount']])
print(merged)

          Player  Inherited Pick   Opp  GP Passing_Yards Passing_TD  \
0  George Kittle             5.4  @SEA  15             -          -   

  Passing_Int Rushing_Yards  Projected_Points Position  ...   13    14    15  \
0           -             -             207.0       TE  ...  1.7  21.1  10.1   

     16    17   18   AVG    TTL PosRank PosCount  
0  18.6  19.2  4.7  15.8  236.6     3.0       40  

[1 rows x 38 columns]


In [35]:
#Need to calculate position scarcity

def calculate_weights(df):
    df['smooth_decay'] = 1 / (df['ADP'] ** 0.5).replace(0, 1e-10)
    df['weighted_gap'] = (df['Inherited Pick'] - df['ADP']) * df['smooth_decay']
    df['scarcity_weight'] = 1 + (1 - (df['PosRank'] - 1) / df['PosCount'].replace(0, 1))

    def fluid_position_weight(row):
        pos = str(row['Position']).strip()
        adp = row['ADP']
        if pos == 'RB':
            return 1 / (1 + np.exp((adp - 2.0) * 2))  # Peak at 2.0
        elif pos == 'WR':
            return 1 / (1 + np.exp((adp - 3.0) * 1.5))  # Softer decay
        elif pos == 'TE':
            return np.exp(-((adp - 3.5) ** 2) / 6)  # Wider bell curve
        elif pos == 'QB':
            return np.exp(-((adp - 5.0) ** 2) / 8)  # Later peak
        elif pos in ['DEF', 'K']:
            return 1 / (1 + np.exp(-(adp - 10)))  # Push to later rounds
        return 0.8

    df['position_weight'] = df.apply(fluid_position_weight, axis=1)
    return df

merged = calculate_weights(merged)

In [36]:
# Calculate total value
merged['total_value'] = (merged['Projected_Points'] *
                         merged['weighted_gap'] *
                         merged['scarcity_weight'] *
                         merged['position_weight'])


In [37]:
# Sharpe Ratio Calculation
weekly_cols = [col for col in merged.columns if col.isdigit()]
sharpe_data = []

for player in input_data['Player']:
    player_mask = merged['Player'].str.strip() == player.strip()
    player_data = merged[player_mask]

    if player_data.empty:
        sharpe_data.append({'Player': player, 'Sharpe': np.nan})
        continue

    try:
        weekly_points = pd.to_numeric(player_data[weekly_cols].iloc[0], errors='coerce').dropna()
        if len(weekly_points) < 2:
            sharpe_data.append({'Player': player, 'Sharpe': np.nan})
            continue

        std_dev = weekly_points.std()
        sharpe = player_data['Projected_Points'].iloc[0] / std_dev if std_dev > 0 else np.nan
        sharpe_data.append({'Player': player, 'Sharpe': sharpe})
    except:
        sharpe_data.append({'Player': player, 'Sharpe': np.nan})


In [38]:
# Merge Sharpe data
sharpe_df = pd.DataFrame(sharpe_data)
merged = merged.merge(sharpe_df, on='Player', how='left')


In [39]:
# Handle missing Sharpe values
merged['Sharpe'] = merged['Sharpe'].fillna(merged['Sharpe'].min())

In [40]:
# Calculate final keeper value
merged['keeper_value'] = 0.75 * merged['total_value'] + 0.25 * merged['Sharpe']

# Scale to [-1, 1]
#min_k = merged['keeper_value'].min()
#max_k = merged['keeper_value'].max()
#merged['keeper_value'] = 2 * (merged['keeper_value'] - min_k) / (max_k - min_k) - 1


In [41]:
# Clean output
print("\nTop Keeper Values:")
print(merged[['Player', 'keeper_value']]
      .sort_values('keeper_value', ascending=False)
      .round(3)
      .to_string(index=False))
print(merged[['Player', 'Sharpe']]
      .sort_values('Sharpe', ascending=False)
      .round(3)
      .to_string(index=False))


Top Keeper Values:
       Player  keeper_value
George Kittle        70.095
       Player  Sharpe
George Kittle  30.156
